In [1]:
import os

In [24]:
len(os.listdir(os.getcwd() + "/data/train/nothing"))

2432

In [6]:
def get_folder_size(folder_path):
    total_size = 0
    try:
        # 遍历文件夹中的所有文件和子文件夹
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total_size += os.path.getsize(filepath)

        # 将字节数转换为更大单位
        total_size_kb = total_size / 1024.0
        total_size_mb = total_size_kb / 1024.0
        total_size_gb = total_size_mb / 1024.0

        print(f"文件夹 '{folder_path}' 的总大小：")
        print(f"{total_size} 字节")
        print(f"{total_size_kb:.2f} KB")
        print(f"{total_size_mb:.2f} MB")
        print(f"{total_size_gb:.2f} GB")

    except FileNotFoundError:
        print(f"文件夹 '{folder_path}' 不存在。")

# 替换为你要检查的文件夹路径
folder_path_to_check = os.getcwd() + "/data/train/nothing"

# 调用函数
get_folder_size(folder_path_to_check)


文件夹 '/mnt/workspace/Baseline/data/train/nothing' 的总大小：
9951349 字节
9718.11 KB
9.49 MB
0.01 GB


In [2]:
from load_lyh_data import *
raw_data_dict = read_folder_npy_data(selected_file=["nothing_processed.npy"])

Loaded nothing_processed.npy: Shape=(15, 300000), Dtype=float64


In [6]:
trial0 = raw_data_dict["nothing_processed.npy"][0:-2,0:1000]

In [7]:
trial1 = raw_data_dict["nothing_processed.npy"][0:-2, 1000:2000]

In [10]:
trial0.mean(axis=1), trial1.mean(axis=1)

(array([ 0.01442085, -1.56067872, -0.61960121, -1.33475331, -0.1442223 ,
        -0.11273622,  0.32821187, -0.00432148,  0.68648067,  1.03092712,
         1.53480116, -0.03937722,  2.97759998]),
 array([-0.44633239,  3.17614217, -0.49152672,  1.46486502,  0.68273147,
        -0.06183269, -0.68817793, -1.48605677, -1.58720262, -2.11189917,
        -3.03902474, -1.33781629, -4.38515012]))

In [4]:
from feature_engineering import *
from load_lyh_data import  *

raw_data_dict = read_folder_npy_data()
standard_input_data_list = pre_preparation(raw_data_dict, config)
preprocessed_data = pre_processing(standard_input_data_list, config)
dataset_np = np.array(preprocessed_data)
dataset_np = dataset_np.reshape(-1, 14, 500)
label_list = [0] * 300 + [1] * 300 + [2] * 300 + [3] * 300
data_df = pd.DataFrame({'raw data': list(dataset_np), 'label': label_list})
data_df = time_domain_feature(data_df)
print('时域提取：')
print(data_df.info())
data_df = freq_domain_feature(data_df)
print('频域提取：')
print(data_df.info())

Loaded nothing1_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing2_orginal.npy: Shape=(100,), Dtype=object
Loaded nothing3_orginal.npy: Shape=(100,), Dtype=object
Loaded left1_orginal.npy: Shape=(100,), Dtype=object
Loaded left2_orginal.npy: Shape=(100,), Dtype=object
Loaded left3_orginal.npy: Shape=(100,), Dtype=object
Loaded right1_orginal.npy: Shape=(100,), Dtype=object
Loaded right2_orginal.npy: Shape=(100,), Dtype=object
Loaded right3_orginal.npy: Shape=(100,), Dtype=object
Loaded leg1_orginal.npy: Shape=(100,), Dtype=object
Loaded leg2_orginal.npy: Shape=(100,), Dtype=object
Loaded leg3_orginal.npy: Shape=(100,), Dtype=object
时域提取：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 72 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   raw data                1200 non-null   object 
 1   label                   1200 non-null   int64  
 2   zero_crossing_rate_c1   1

/home/pai/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 500, using nperseg = 500
  warnings.warn('nperseg = {0:d} is greater than input length '


频域提取：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Columns: 142 entries, raw data to gamma_c14
dtypes: float64(140), int64(1), object(1)
memory usage: 1.3+ MB
None


/mnt/workspace/Baseline/feature_engineering.py:282: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/mnt/workspace/Baseline/feature_engineering.py:282: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eeg_dataset[f'{freq_names[j]}_c{i+1}'] = np.array(energy)[:,i*nFreqs+j]
/mnt/workspace/Baseline/feature_engineering.py:282: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [5]:
from lightgbm import LGBMClassifier
# 构造数据集
from sklearn.model_selection import train_test_split

# 打乱数据集
shuffled_df = data_df.sample(frac=1, random_state=42)

# 划分训练集和测试集
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)
y_train = train_df['label']
X_train = train_df.drop(columns=['label', 'raw data'])
y_test = test_df['label']
X_test = test_df.drop(columns=['label', 'raw data'])


lgm = LGBMClassifier()
lgm.fit(X_train, y_train)

# 手工特征提取+未调参LightGBM
y_train_pred = lgm.predict(X_train)
acc_train = (y_train == y_train_pred).sum() / len(y_train)

y_test_pred = lgm.predict(X_test)
acc_test = (y_test == y_test_pred).sum() / len(y_test)
acc_train, acc_test

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001760 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33576
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 140
[LightGBM] [Info] Start training from score -1.382136
[LightGBM] [Info] Start training from score -1.365675
[LightGBM] [Info] Start training from score -1.361602
[LightGBM] [Info] Start training from score -1.437588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

(1.0, 0.9125)

In [8]:
import numpy as np

def test(data):
    data = data[::2].copy()
    return data

d = np.array([1, 2, 3, 4])
e = test(d)

In [9]:
e[0] = 2
e

array([2, 3])

In [10]:
d

array([1, 2, 3, 4])